In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial, reduce
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
exchange = pd.read_csv('Exchange_rate.csv')
gdp_dollar = pd.read_csv('GDP_dollar_new.csv',encoding='latin-1') # 2020'e kadar olan veri ile değiştirilecek
infl = pd.read_csv('Inflation.csv',encoding='latin-1')
lending = pd.read_csv('Lending_rate.csv',encoding='latin-1')
stock_marget = pd.read_csv('Stock_market_change.csv')
crises = pd.read_csv('Country_crises_new.csv',encoding='latin-1')
crises.columns = ['country','Banking_Crisis','Currency_Crisis','Sovereign_Crisis','Sovereign_Restructuring','Covid']

df_list = [exchange,gdp_dollar,infl,lending,stock_marget]
for data in df_list:
    data.drop(columns=['country_code'],inplace=True)

In [4]:
# Exchange_rate first process
exchange_v0= pd.melt(exchange,id_vars =['country'],value_vars =list(exchange.columns),
            var_name='years',value_name='exchange_year_value')
exchange_v0.isnull().any()

country                False
years                  False
exchange_year_value    False
dtype: bool

In [5]:
# gdp_dollar first process
gdp_dollar_v0 = pd.melt(gdp_dollar,id_vars =['country'],value_vars =list(gdp_dollar.columns),
            var_name='years',value_name='gdp_dollar_year_value')
gdp_dollar_v0.isnull().any()

country                  False
years                    False
gdp_dollar_year_value    False
dtype: bool

In [6]:
# infl first process
infl_v0 = pd.melt(infl,id_vars =['country'],value_vars =list(infl.columns),
            var_name='years',value_name='infl_year_value')
print(infl_v0.isnull().any())

country            False
years              False
infl_year_value     True
dtype: bool


In [7]:
# infl first process
lending_v0 = pd.melt(lending,id_vars =['country'],value_vars =list(lending.columns),
            var_name='years',value_name='lending_year_value')
print(lending_v0.isnull().any())

country               False
years                 False
lending_year_value     True
dtype: bool


In [7]:
# stock_marget first process
stock_marget_v0 = pd.melt(stock_marget,id_vars =['country'],value_vars =list(stock_marget.columns),
            var_name='years',value_name='stock_marget_year_value')
print(stock_marget_v0.isnull().any())

country                    False
years                      False
stock_marget_year_value     True
dtype: bool


In [8]:
merged_df = [exchange_v0,gdp_dollar_v0,infl_v0,lending_v0,stock_marget_v0]
merge = partial(pd.merge,how='left')
df = reduce(merge,merged_df)

In [9]:
# Type dönüşümü için gerekli
df = df.apply(lambda x: x.replace('..',np.NaN))

type_convert = ['exchange_year_value','gdp_dollar_year_value']
for type_ in type_convert:
    df[type_] = df[type_].astype('float64')

In [10]:
df

,country,years,exchange_year_value,gdp_dollar_year_value,infl_year_value,lending_year_value,stock_marget_year_value
0,Afghanistan,1960,45.0000,NaN,NaN,NaN,NaN
1,Albania,1960,NaN,NaN,NaN,NaN,NaN
2,Algeria,1960,4.9371,2.744287e+10,NaN,NaN,NaN
3,American Samoa,1960,1.0000,NaN,NaN,NaN,NaN
4,Andorra,1960,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
13232,Virgin Islands (U.S.),2020,NaN,NaN,NaN,NaN,NaN
13233,West Bank and Gaza,2020,1.0000,1.224375e+10,NaN,NaN,NaN
13234,"Yemen, Rep.",2020,743.0000,NaN,NaN,NaN,NaN
13235,Zambia,2020,18.3441,2.864814e+10,NaN,NaN,NaN


In [11]:
#Ülkelerin growth_rate hesaplanması
df.sort_values(by=['country','years'],inplace=True,ignore_index=True)

def calculate_growth_rate(df,col):
    return df[col].pct_change().mul(100)
exchange_growth_rate = df.groupby('country').apply(calculate_growth_rate,col='exchange_year_value')
gdp_dollar_growth_rate      = df.groupby('country').apply(calculate_growth_rate,col='gdp_dollar_year_value')

exchange_growth_rate = exchange_growth_rate.to_frame('exchange_growth_rate').reset_index(level=0)
gdp_dollar_growth_rate = gdp_dollar_growth_rate.to_frame('gdp_dollar_growth_rate').reset_index(level=0)

df = pd.concat([df,exchange_growth_rate.iloc[:,1],gdp_dollar_growth_rate.iloc[:,1]],axis=1)

In [12]:
#growth rate oranı hesaplandıktan sonra usa verileri ekleniyor
usa = df[df["country"] == "United States"]
usa.columns = [f'usa_{i}' if i not in ['country', 'years'] else f'{i}' for i in df.columns]

df_v0 = pd.merge(df,usa[['years','usa_exchange_year_value','usa_gdp_dollar_year_value','usa_infl_year_value',
                         'usa_lending_year_value','usa_stock_marget_year_value',
                         'usa_exchange_growth_rate','usa_gdp_dollar_growth_rate']],how='left',on='years')
df_v0.drop(df_v0.loc[df_v0['country'] == 'United States'].index,inplace=True)

In [13]:
#Kriz verisi içinde country kolonunda boşlukar var. onları temizliyorum
crises['country'] = crises['country'].str.replace(' ','')

In [14]:
#Kriz verisi ile df_0 verisi birleştirirken df_0 verisine göre birleştiriyorum. O yüzden krizde ki isimleri df_0'a benzettim
replacers = {'BosniaandHerzegovina':'Bosnia and Herzegovina',
            'Brunei':'Brunei Darussalam',
            'BurkinaFaso':'Burkina Faso',
            'CentralAfricanRep':'Central African Republic',
            'China,P.R.':'China',
            'Congo,Dem.Rep.of':'Congo, Dem. Rep.',
            'Congo,Rep.of':'Congo, Rep',
            'CostaRica':'Costa Rica',
            'Côted\x92Ivoire':"Cote d'Ivoire",
            'CzechRepublic':'Czech Republic',
            'Egypt':'Egypt, Arab Rep.',
            'ElSalvador':'El Salvador',
            'EquatorialGuinea':'Equatorial Guinea',
            'Gambia,The':'Gambia, The',
            'China,P.R.:HongKong':'Hong Kong SAR, China',
            'Iran,I.R.of':'Iran, Islamic Rep.',
            'Korea':'Korea, Rep.',
            'KyrgyzRepublic':'Kyrgyz Republic',
            'LaoPeople\x92sDem.Rep.':'Lao PDR',
            'NewCaledonia':'New Caledonia',
            'NewZealand':'New Zealand',
            'PapuaNewGuinea':'Papua New Guinea',
            'Russia':'Russian Federation',
            'SãoToméandPrincipe':'Sao Tome and Principe',
            'Serbia,Republicof':'Serbia',
            'SierraLeone':'Sierra Leone',
            'SlovakRepublic':'Slovak Republic',
            'SouthAfrica':'South Africa',
            'SouthSudan':'South Sudan',
            'SriLanka':'Sri Lanka',
            'SyrianArabRepublic':'Syrian Arab Republic',
            'TrinidadandTobago':'Trinidad and Tobago',
            'UnitedKingdom':'United Kingdom',
            'Venezuela':'Venezuela, RB',
            'Yemen':'Yemen, Rep'}

crises['country'] = crises['country'].replace(replacers)

In [15]:
#Criss datası ile birleştirme 
df= pd.merge(df_v0,crises,how='left',on='country')
df['Banking_Crisis'] = df['Banking_Crisis'].str.replace(' ','')
df.head()

,country,years,exchange_year_value,gdp_dollar_year_value,infl_year_value,lending_year_value,stock_marget_year_value,exchange_growth_rate,gdp_dollar_growth_rate,usa_exchange_year_value,usa_gdp_dollar_year_value,usa_infl_year_value,usa_lending_year_value,usa_stock_marget_year_value,usa_exchange_growth_rate,usa_gdp_dollar_growth_rate,Banking_Crisis,Currency_Crisis,Sovereign_Crisis,Sovereign_Restructuring,Covid
0,Afghanistan,1960,45.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.175860e+12,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1961,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.248900e+12,1.0,5.0,18.67,0.0,2.299849,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1962,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.447090e+12,1.0,5.0,-5.96,0.0,6.100219,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1963,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.598760e+12,1.0,5.0,12.10,0.0,4.399943,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1964,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.807490e+12,1.0,5.0,16.48,0.0,5.800053,NaN,NaN,NaN,NaN,NaN


In [16]:
#Criss datası içinde bulunan değişkenler ile dummy üretme

Banking_Crisis_index = []
for i, (year, cris) in enumerate(zip(df['years'], df['Banking_Crisis'])):
    if str(year) in str(cris):
        Banking_Crisis_index.append(i)
        
df["Banking_Crisis_dummy"] = 0
df["Banking_Crisis_dummy"].loc[Banking_Crisis_index] = 1
df["Banking_Crisis_dummy"] = df["Banking_Crisis_dummy"].astype('category')


#Currency_Crisis
Currency_Crisis_index = []
for i, (year, cris) in enumerate(zip(df['years'], df['Currency_Crisis'])):
    if str(year) in str(cris):
        Currency_Crisis_index.append(i)
        
df["Currency_Crisis_dummy"] = 0
df["Currency_Crisis_dummy"].loc[Currency_Crisis_index] = 1
df["Currency_Crisis_dummy"] = df["Currency_Crisis_dummy"].astype('category')



#Sovereign_Crisis
Sovereign_Crisis_index = []
for i, (year, cris) in enumerate(zip(df['years'], df['Sovereign_Crisis'])):
    if str(year) in str(cris):
        Sovereign_Crisis_index.append(i)
        
df["Sovereign_Crisis_dummy"] = 0
df["Sovereign_Crisis_dummy"].loc[Sovereign_Crisis_index] = 1
df["Sovereign_Crisis_dummy"] = df["Sovereign_Crisis_dummy"].astype('category')


#Covid dummy
Covid_index = []
for i, (year, cris) in enumerate(zip(df['years'], df['Covid'])):
    if str(year) in str(cris):
        Covid_index.append(i)
        
df["Covid_dummy"] = 0
df["Covid_dummy"].loc[Covid_index] = 1
df["Covid_dummy"] = df["Covid_dummy"].astype('category')


#dummy için kullanılan değişkenleri sil
df.drop(columns=['Banking_Crisis','Currency_Crisis','Sovereign_Crisis','Sovereign_Restructuring','Covid'],inplace=True)

In [17]:
df.head()

,country,years,exchange_year_value,gdp_dollar_year_value,infl_year_value,lending_year_value,stock_marget_year_value,exchange_growth_rate,gdp_dollar_growth_rate,usa_exchange_year_value,usa_gdp_dollar_year_value,usa_infl_year_value,usa_lending_year_value,usa_stock_marget_year_value,usa_exchange_growth_rate,usa_gdp_dollar_growth_rate,Banking_Crisis_dummy,Currency_Crisis_dummy,Sovereign_Crisis_dummy,Covid_dummy
0,Afghanistan,1960,45.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.175860e+12,2.0,5.0,NaN,NaN,NaN,0,0,0,0
1,Afghanistan,1961,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.248900e+12,1.0,5.0,18.67,0.0,2.299849,0,0,0,0
2,Afghanistan,1962,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.447090e+12,1.0,5.0,-5.96,0.0,6.100219,0,0,0,0
3,Afghanistan,1963,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.598760e+12,1.0,5.0,12.10,0.0,4.399943,0,0,0,0
4,Afghanistan,1964,45.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,3.807490e+12,1.0,5.0,16.48,0.0,5.800053,0,0,0,0


In [18]:
df.to_csv('all_data_merged_v0.csv',index=False)